In [1]:
%pip install pandas_datareader pykrx yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 642.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.9 MB/s eta 0:00:00


# src

## common

### path.py

In [10]:
from pandas import Series
import os


class PATH:
    try:
        ROOT = os.path.dirname(__file__)
        while not ROOT.endswith('yuho'):
            ROOT = os.path.dirname(ROOT)
    except NameError:
        ROOT = 'https://raw.githubusercontent.com/labwons/yuho/main/'

    try:
        DESKTOP = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
        DOWNLOADS = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    except KeyError:
        DESKTOP = DOWNLOADS = ROOT

    BASE   = os.path.join(ROOT, r'src/fetch/market/json/baseline.json')
    GROUP  = os.path.join(ROOT, r'src/fetch/market/json/group.json')
    INDEX  = os.path.join(ROOT, r'src/fetch/market/json/index.json')
    SPEC   = os.path.join(ROOT, r'src/fetch/market/json/spec.json')
    STATE  = os.path.join(ROOT, r'src/fetch/market/json/state.json')

    DOCS   = os.path.join(ROOT, r'docs')
    BUBBLE = os.path.join(ROOT, r'docs/src/json/bubble.json')
    MAP    = os.path.join(ROOT, r'docs/src/json/treemap.json')
    MACRO  = os.path.join(ROOT, r'docs/src/json/macro.json')

    # HTML = html(ROOT)
    HTML = Series()
    HTML.MAP = os.path.join(ROOT, r'docs/index.html')
    HTML.BUBBLE = os.path.join(ROOT, r'docs/bubble/index.html')
    HTML.MACRO = os.path.join(ROOT, r'docs/macro/index.html')
    HTML.TEMPLATES = os.path.join(ROOT, r'src/render/templates')

    JS = Series()
    JS.MAP = os.path.join(ROOT, r'docs/src/js/marketmap.js')





if __name__ == "__main__":
    # print(PATH.ROOT)
    # print(PATH.GROUP)
    # print(PATH.STATE)
    # print(PATH.SPEC)
    # print(PATH.INDEX)
    # print(PATH.MAP)
    print(PATH.HTML.MAP)


https://raw.githubusercontent.com/labwons/yuho/main/docs/index.html


## fetch

### market

#### group.py

In [2]:
from pandas import (
    DataFrame,
    concat,
    Index,
    read_json,
    Series
)
from pykrx.stock import get_index_portfolio_deposit_file
from re import compile
from requests import get
from requests.exceptions import JSONDecodeError
from time import sleep, time
from typing import (
    Dict,
    Iterable,
    List
)
if "PATH" not in globals():
    try:
        from ...common.path import PATH
    except ImportError:
        from src.common.path import PATH


SECTOR_CODE:Dict[str, str] = {
    'WI100': '에너지', 'WI110': '화학',
    'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
    'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
    'WI400': '필수소비재', 'WI410': '건강관리',
    'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
    'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
    'WI700': '통신서비스',
    'WI800': '유틸리티'
}
CODE_LABEL:Dict[str, str] = {
    'CMP_CD': 'ticker', 'CMP_KOR': 'name',
    'SEC_CD': 'sectorCode', 'SEC_NM_KOR': 'sectorName',
    'IDX_CD': 'industryCode', 'IDX_NM_KOR': 'industryName',
}
REITS_CODE:Dict[str, str] = {
    "088980": "맥쿼리인프라",
    "395400": "SK리츠",
    "365550": "ESR켄달스퀘어리츠",
    "330590": "롯데리츠",
    "348950": "제이알글로벌리츠",
    "293940": "신한알파리츠",
    "432320": "KB스타리츠",
    "094800": "맵스리얼티1",
    "357120": "코람코라이프인프라리츠",
    "448730": "삼성FN리츠",
    "451800": "한화리츠",
    "088260": "이리츠코크렙",
    "334890": "이지스밸류리츠",
    "377190": "디앤디플랫폼리츠",
    "404990": "신한서부티엔디리츠",
    "417310": "코람코더원리츠",
    "400760": "NH올원리츠",
    "350520": "이지스레지던스리츠",
}


class MarketGroup(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.GROUP, orient='index'))
            self.index = self.index.astype(str).str.zfill(6)
            self.index.name = 'ticker'
            return

        date = self.fetchTradingDate()
        self.log = f'Begin [Market Group Fetch] @{date}'
        objs, size = [], len(SECTOR_CODE) + 1
        for n, (code, name) in enumerate(SECTOR_CODE.items()):
            obj = self.fetchWiseGroup(code, date)
            objs.append(obj)
            proc = f"... {str(n + 1).zfill(2)} / {size} : {code} {name} :: "
            self.log = f"{proc}Fail" if obj.empty else f"{proc}Success"

        reits = DataFrame(data={'CMP_KOR': REITS_CODE.values(), 'CMP_CD':REITS_CODE.keys()})
        reits[['SEC_CD', 'IDX_CD', 'SEC_NM_KOR', 'IDX_NM_KOR']] \
              = ['G99', 'WI999', '리츠', '리츠']
        objs.append(reits)

        self.log = f"... {size} / {size} : WI999 리츠 :: Success"

        super().__init__(concat(objs, axis=0, ignore_index=True))
        self.drop(columns=[key for key in self if not key in CODE_LABEL], inplace=True)
        self.drop(index=self[self['SEC_CD'].isna()].index, inplace=True)
        self.rename(columns=CODE_LABEL, inplace=True)
        self.set_index(keys="ticker", inplace=True)
        self['industryName'] = self['industryName'].str.replace("WI26 ", "")

        kq, lg = self.fetchKosdaqList(self.index), self.fetchLargeCapList(self.index)
        self.loc[kq, 'name'] = self.loc[kq, 'name'] + '*'
        self.loc[lg, 'stockSize'] = 'large'
        self.log = "... Identify Tickers: Success"

        sc_mdi = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G50')].index
        sc_edu = self[(self['industryCode'] == 'WI330') & (self['sectorCode'] == 'G25')].index
        sc_sw = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G50')].index
        sc_it = self[(self['industryCode'] == 'WI600') & (self['sectorCode'] == 'G45')].index
        self.loc[sc_mdi, 'industryCode'], self.loc[sc_mdi, 'industryName'] = 'WI331', '미디어'
        self.loc[sc_edu, 'industryCode'], self.loc[sc_edu, 'industryName'] = 'WI332', '교육'
        self.loc[sc_sw, 'industryCode'], self.loc[sc_sw, 'industryName'] = 'WI601', '소프트웨어'
        self.loc[sc_it, 'industryCode'], self.loc[sc_it, 'industryName'] = 'WI602', 'IT서비스'

        self.log = f'End [Market Group Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchTradingDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseGroup(cls, code:str, date:str="", countdown:int=5) -> DataFrame:

        resp = get(f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={date}&sec_cd={code}')
        try:
            return DataFrame(resp.json()['list'])
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseGroup(code, date, countdown - 1)

    @classmethod
    def fetchKosdaqList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2001')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers

    @classmethod
    def fetchLargeCapList(cls, _tickers:Index=None) -> List[str]:
        tickers = get_index_portfolio_deposit_file('2203') \
                + get_index_portfolio_deposit_file('1028')
        if not _tickers.empty:
            tickers = [ticker for ticker in _tickers if ticker in tickers]
        return tickers

if __name__ == "__main__":
    marketGroup = MarketGroup(True)
    # print(marketGroup)
    print(marketGroup.log)

# date = MarketGroup.fetchTradingDate()
# print(date)
# for cd, name in SECTOR_CODE.items():
#     print(cd, name, end= " ")
#     resp = get(
#         f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={date}&sec_cd={cd}'
#     )
#     print(resp)

Begin [Market Group Fetch] @20250305
... 01 / 27 : WI100 에너지 :: Success
... 02 / 27 : WI110 화학 :: Success
... 03 / 27 : WI200 비철금속 :: Success
... 04 / 27 : WI210 철강 :: Success
... 05 / 27 : WI220 건설 :: Success
... 06 / 27 : WI230 기계 :: Success
... 07 / 27 : WI240 조선 :: Success
... 08 / 27 : WI250 상사,자본재 :: Success
... 09 / 27 : WI260 운송 :: Success
... 10 / 27 : WI300 자동차 :: Success
... 11 / 27 : WI310 화장품,의류 :: Success
... 12 / 27 : WI320 호텔,레저 :: Success
... 13 / 27 : WI330 미디어,교육 :: Success
... 14 / 27 : WI340 소매(유통) :: Success
... 15 / 27 : WI400 필수소비재 :: Success
... 16 / 27 : WI410 건강관리 :: Success
... 17 / 27 : WI500 은행 :: Success
... 18 / 27 : WI510 증권 :: Success
... 19 / 27 : WI520 보험 :: Success
... 20 / 27 : WI600 소프트웨어 :: Success
... 21 / 27 : WI610 IT하드웨어 :: Success
... 22 / 27 : WI620 반도체 :: Success
... 23 / 27 : WI630 IT가전 :: Success
... 24 / 27 : WI640 디스플레이 :: Success
... 25 / 27 : WI700 통신서비스 :: Success
... 26 / 27 : WI800 유틸리티 :: Success
... 27 / 27 : WI999 리츠 :: Success

#### index.py

In [3]:
from pandas import (
    DataFrame,
    read_json,
    to_datetime
)
from pykrx.stock import (
    get_index_ohlcv_by_date,
    get_nearest_business_day_in_a_week
)
from re import compile, search
from requests import get
from requests.exceptions import JSONDecodeError
from time import sleep, time
from typing import (
    Dict,
    List
)
from warnings import simplefilter
if "PATH" not in globals():
    try:
        from ...common.path import PATH
    except ImportError:
        from src.common.path import PATH

simplefilter(action='ignore', category=FutureWarning)


INDEX_CODE:Dict[str, str] = {
    '1001': 'KOSPI', '2001': 'KOSDAQ',
    'WI100': '에너지', 'WI110': '화학',
    'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
    'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
    'WI400': '필수소비재', 'WI410': '건강관리',
    'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
    'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
    'WI700': '통신서비스',
    'WI800': '유틸리티'
}


class MarketIndex(DataFrame):

    _log:List[str] = []

    def __init__(self, update:bool=True):
        stime = time()
        super().__init__(read_json(PATH.INDEX, orient='index'))
        self.index = self.index.date
        if not update:
            return

        trading_date = get_nearest_business_day_in_a_week()
        server_date = self.fetchServerDate()
        self.log = f'Begin [Market Index Fetch] @{trading_date}'

        for n, (code, name) in enumerate(INDEX_CODE.items()):
            proc = f'... ({n + 1} / {len(INDEX_CODE)}) : {code} {name} :: '
            start = self[code].dropna().index[-1]
            end = trading_date if code in ['1001', '2001'] else server_date
            if start == end:
                continue
            fetch = self.fetchWiseSeries(code, f'{start}', f'{end}')
            if fetch.empty:
                self.log = f'{proc}Fail'
                continue
            for dt in fetch.index:
                self.at[dt, code] = fetch.loc[dt, code]
            self.log = f'{proc}Success '
        self.log = f'End [Market Index Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def _netDate2normDate(cls, timestamp:str):
        timestamp = int(search(r'\((\d+)\)', timestamp).group(1))
        return to_datetime(timestamp, unit='ms', utc=True) \
               .tz_convert('Asia/Seoul') \
               .date()

    @classmethod
    def fetchServerDate(cls) -> str:
        URL = 'https://www.wiseindex.com/Index/Index#/G1010.0.Components'
        pattern = compile(r"var\s+dt\s*=\s*'(\d{8})'")
        return pattern.search(get(URL).text).group(1)

    @classmethod
    def fetchWiseSeries(cls, code:str, start:str, end:str, countdown:int=5) -> DataFrame:
        if code in ['1001', '2001']:
            fetch = get_index_ohlcv_by_date(start, end, code, 'd', False)
            fetch.index = fetch.index.date
            fetch = fetch.rename(columns={"종가": code})
            return fetch

        resp = get(f'http://www.wiseindex.com/DataCenter/GridData?currentPage=1&endDT={end}&fromDT={start}&index_ids={code}&isEnd=1&itemType=1&perPage=10000&term=1')
        try:
            fetch = DataFrame(resp.json())[["TRD_DT", "IDX1_VAL1"]]
            fetch["TRD_DT"] = fetch["TRD_DT"].apply(cls._netDate2normDate)
            return fetch.rename(columns={"IDX1_VAL1": code}).set_index(keys="TRD_DT")
        except JSONDecodeError:
            if countdown == 0:
                return DataFrame()
            sleep(5)
            return cls.fetchWiseSeries(code, start, end, countdown - 1)


# if __name__ == "__main__":
#     marketIndex = MarketIndex(True)
#     # print(marketIndex)
#     print(marketIndex.log)


#### state.py

In [23]:
from datetime import datetime, timedelta
from io import StringIO
from pandas import (
    concat,
    DataFrame,
    Index,
    read_html,
    read_json,
)
from pykrx.stock import (
    get_exhaustion_rates_of_foreign_investment,
    get_nearest_business_day_in_a_week,
    get_market_cap_by_ticker,
    get_market_fundamental,
    get_market_ohlcv_by_date
)
from requests import get
from requests.exceptions import JSONDecodeError, SSLError
from time import time
from typing import Dict, Iterable, List

if "PATH" not in globals():
    try:
        from ...common.path import PATH
    except ImportError:
        from src.common.path import PATH

IPO_LABEL: Dict[str, str] = {
    '회사명': 'name', '종목코드': 'ticker',
    '상장일': 'ipo', '주요제품': 'products', '결산월': 'settlementMonth'
}
CAP_LABEL: Dict[str, str] = {
    '종가': 'close', '시가총액': 'marketCap',
    '거래량': 'volume', '거래대금': 'amount', '상장주식수': 'shares'
}
MUL_LABEL: Dict[str, str] = {
    'PER': 'PER', 'PBR': 'PBR', 'DIV': 'dividendYield'
}
PCT_LABEL: Dict[str, str] = {"지분율": 'foreignRate'}
PRC_LABEL: Dict[str, str] = {
    "시가": "open", "고가": "high", "저가": "low", "종가": "close",
    "거래량": "volume", "거래대금": "amount"
}
INTERVALS: Dict[str, int] = {
    'D+0': 0, 'D-1': 1, 'W-1': 7,
    'M-1': 30, 'M-3': 91, 'M-6': 182, 'Y-1': 365
}


class MarketState(DataFrame):
    _log: List[str] = []

    def __init__(self, update: bool = True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.STATE, orient='index'))
            self.index = self.index.astype(str).str.zfill(6)
            return

        date = get_nearest_business_day_in_a_week()
        self.log = f'Begin [Market State Fetch] @{date}'

        fdef = [self.fetchMarketCap, self.fetchMultiples, self.fetchForeignRate]
        ks = concat([func(date, 'KOSPI') for func in fdef], axis=1)
        ks['market'] = 'kospi'
        self.log = f'... Fetch KOSPI Market State :: {"Fail" if ks.empty else "Success"}'
        kq = concat([func(date, 'KOSDAQ') for func in fdef], axis=1)
        kq['market'] = 'kosdaq'
        self.log = f'... Fetch KOSDAQ Market State :: {"Fail" if kq.empty else "Success"}'
        market = concat([ks, kq], axis=0)

        market = market[
            (~market.index.isin(self.fetchKonexList(date))) &
            (market.index.isin(self.fetchIpoList().index)) &
            (~market['shares'].isna())
            ]
        market = market[market['marketCap'] >= market['marketCap'].median()]

        returns = self.fetchReturns(date, market.index)
        self.log = f'... Fetch Returns :: {"Fail" if returns.empty else "Success"}'

        merge = returns.join(market, how='left')
        merge = merge.sort_values(by='marketCap', ascending=False)
        merge["date"] = datetime.strptime(date, "%Y%m%d").strftime("%Y/%m/%d")
        super().__init__(merge)

        self.log = f'End [Market State Fetch] / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

    @classmethod
    def fetchKonexList(cls, date: str) -> Index:
        try:
            return get_market_cap_by_ticker(date=date, market='KONEX').index
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return Index([])

    @classmethod
    def fetchIpoList(cls) -> DataFrame:
        _url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download'
        try:
            resp = StringIO(get(_url).text)
            df = read_html(io=resp, encoding='euc-kr')[0][IPO_LABEL.keys()] \
                .rename(columns=IPO_LABEL) \
                .set_index(keys='ticker')
            df.index = df.index.astype(str).str.zfill(6)
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(IPO_LABEL.values()))

    @classmethod
    def fetchMarketCap(cls, date: str, market: str = 'ALL') -> DataFrame:
        try:
            df = get_market_cap_by_ticker(date=date, market=market, alternative=True) \
                .rename(columns=CAP_LABEL)
            df.index.name = 'ticker'
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(CAP_LABEL.values()))

    @classmethod
    def fetchMultiples(cls, date: str, market: str = 'ALL') -> DataFrame:
        try:
            df = get_market_fundamental(date=date, market=market) \
                .rename(columns=MUL_LABEL)
            df.index.name = "ticker"
            return df[MUL_LABEL.values()]
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(MUL_LABEL.values()))

    @classmethod
    def fetchForeignRate(cls, date: str, market: str = 'ALL') -> DataFrame:
        try:
            df = get_exhaustion_rates_of_foreign_investment(date=date, market=market) \
                .rename(columns=PCT_LABEL)
            df.index.name = 'ticker'
            return round(df[PCT_LABEL.values()].astype(float), 2)
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(PCT_LABEL.values()))

    @classmethod
    def fetchReturns(cls, date: str, tickers: Iterable = None) -> DataFrame:
        tdate = datetime.strptime(date, "%Y%m%d")
        intv = {key: tdate - timedelta(val) for key, val in INTERVALS.items()}

        objs = {
            key: cls.fetchMarketCap(val.strftime("%Y%m%d"))
            for key, val in intv.items()
        }
        base = concat(objs, axis=1)
        if tickers is not None:
            base = base[base.index.isin(tickers)]

        returns = concat({
            dt: base['D+0']['close'] / base[dt]['close']  - 1 for dt in objs
        }, axis=1)
        returns.drop(columns=['D+0'], inplace=True)

        diff = base[base['Y-1']['shares'] != base['D+0']['shares']].index
        fdate = (tdate - timedelta(380)).strftime("%Y%m%d")
        for ticker in diff:
            ohlc = get_market_ohlcv_by_date(fromdate=fdate, todate=date, ticker=ticker)
            for interval in returns.columns:
                ohlc_copy = ohlc[ohlc.index >= intv[interval]]['종가']
                returns.loc[ticker, interval] = ohlc_copy.iloc[-1] / ohlc_copy.iloc[0] - 1
        return round(100 * returns, 2)


if __name__ == "__main__":
    marketState = MarketState(True)
    print(marketState)
    print(marketState.log)



In [12]:
print(marketState)

          D-1    W-1    M-1    M-3    M-6    Y-1    close        marketCap    volume        amount      shares    PER   PBR  dividendYield  foreignRate  market        date
ticker                                                                                                                                                                     
005930   0.56  -3.55   3.04   1.12 -21.30 -25.51    54300  321436339164600  12171691  659824465910  5919637922  25.48  1.04           2.66        50.38   kospi  2025/03/06
000660   0.36   3.53  -0.68 -10.08 -17.15 -15.33   192400  140067655026000   2526406  485275883650   728002365   0.00  2.47           0.62        55.31   kospi  2025/03/06
207940   1.15   2.57  -5.13 -14.69 -17.26 -30.53  1130000   80426620000000     42694   48330534500    71174000  93.77  8.18           0.00        13.56   kospi  2025/03/06
373220   0.00  10.93  -0.30  14.37  22.75  15.87   334000   78156000000000    249402   83760454750   234000000  63.17  3.87           0.00  

In [24]:
tester = MarketState.fetchReturns('20250306')
print(tester)

         D-1   W-1   M-1    M-3    M-6    Y-1
ticker                                       
005930  0.56 -3.55  3.04   1.12 -21.30 -25.51
000660 -0.36 -3.41  0.68  11.21  20.70  18.11
207940 -1.14 -2.50  5.41  17.22  20.86  43.95
373220  0.00 -9.85  0.30 -12.57 -18.54 -13.70
005380  1.49 -1.45 -1.69  -3.37 -13.71 -21.27
...      ...   ...   ...    ...    ...    ...
150440   NaN   NaN   NaN    NaN    NaN    NaN
386580   NaN   NaN   NaN    NaN    NaN    NaN
161570   NaN   NaN   NaN    NaN    NaN    NaN
224020   NaN   NaN   NaN    NaN    NaN    NaN
322190   NaN   NaN   NaN    NaN    NaN    NaN

[2935 rows x 6 columns]


In [ ]:
marketState.to_json(orient='index').replace('nan', '')

#### spec.py

In [5]:
from datetime import datetime
from io import StringIO
from numpy import nan
from pandas import (
    concat,
    DataFrame,
    read_html,
    read_json,
    Series
)
from pykrx.stock import get_market_cap_by_ticker
from re import DOTALL, sub
from requests import get
from requests.exceptions import JSONDecodeError, SSLError
from time import time
from typing import Dict, List, Union, Tuple
from xml.etree.ElementTree import Element, fromstring
if "PATH" not in globals():
    try:
        from ...common.path import PATH
    except ImportError:
        from src.common.path import PATH


CAP_LABEL: Dict[str, str] = {
    '종가': 'close', '시가총액': 'marketCap',
    '거래량': 'volume', '거래대금': 'amount', '상장주식수': 'shares'
}
IPO_LABEL = {
    '회사명': 'name', '종목코드': 'ticker',
    '상장일': 'ipo', '주요제품': 'products', '결산월': 'settlementMonth'
}
OVERVIEW_TAG: Dict[str, str] = {
    'high52': 'price/high52week',
    'low52': 'price/low52week',
    'beta': 'price/beta',
    'floatShares': 'price/ff_sher_rt',
    'estPrice': 'consensus/target_price',
    'estEps': 'consensus/eps'
}
STATEMENT_TAG: Dict[str, str] = {
    'consolidateAnnual': f'financial_highlight_ifrs_D/financial_highlight_annual',
    'consolidateQuarter': f'financial_highlight_ifrs_D/financial_highlight_quarter',
    'separateAnnual': f'financial_highlight_ifrs_B/financial_highlight_annual',
    'separateQuarter': f'financial_highlight_ifrs_B/financial_highlight_quarter'
}


class MarketSpec(DataFrame):
    _log: List[str] = []

    def __init__(self, update: bool = True):
        stime = time()
        if not update:
            super().__init__(read_json(PATH.SPEC, orient='index'))
            self.index = self.index.astype(str).str.zfill(6)
            return

        date = datetime.today().strftime("%Y%m%d")
        self.log = f'Begin [Market Spec Fetch] @{date}'

        market = concat([self.fetchMarketCap(date, 'KOSPI'), self.fetchMarketCap(date, 'KOSDAQ')])
        market = market[
            (market.index.isin(self.fetchIpoList().index)) &
            (~market['shares'].isna())
        ]
        market = market[market['marketCap'] >= market['marketCap'].median()]

        objs = []
        for n, ticker in enumerate(market.index):
            try:
                xml = self.fetchXml(ticker)
                obj = self.fetchOverview(xml)
                obj.name = ticker
                A, Q = self.fetchStatement(xml)
                if A.empty or Q.empty:
                    objs.append(obj)
                    continue

                Aa, Qq = self.customizeStatement(A), self.customizeStatement(Q)
                obj['trailingRevenue'] = Qq.iloc[-1]['trailingRevenue']
                obj['trailingEps'] = Qq.iloc[-1]['trailingEps']
                obj['trailingProfitRate'] = Qq.iloc[-1]['trailingProfitRate']
                obj['averageRevenueGrowth_A'] = Aa['revenueGrowth'].mean()
                obj['averageProfitGrowth_A'] = Aa['profitGrowth'].mean()
                obj['averageEpsGrowth_A'] = Aa['epsGrowth'].mean()
                obj['RevenueGrowth_A'] = Aa.iloc[-1]['revenueGrowth']
                obj['RevenueGrowth_Q'] = Qq.iloc[-1]['revenueGrowth']
                obj['ProfitGrowth_A'] = Aa.iloc[-1]['profitGrowth']
                obj['ProfitGrowth_Q'] = Qq.iloc[-1]['profitGrowth']
                obj['EpsGrowth_A'] = Aa.iloc[-1]['epsGrowth']
                obj['EpsGrowth_Q'] = Qq.iloc[-1]['epsGrowth']

                dividend = Aa['배당수익률(%)'].dropna()
                obj['fiscalDividendYield'] = dividend.values[-1] if not dividend.empty else nan

                debt = Aa['부채비율(%)'].dropna()
                obj['fiscalDebtRatio'] = debt.values[-1] if not debt.empty else nan
                objs.append(obj)
            except Exception as reason:
                self.log = f'... Failed to fetch: {ticker} / {reason}'

        super().__init__(concat(objs, axis=1).T)
        for col in self:
            self[col] = round(self[col], 4 if col == 'beta' else 2)

        self.log = f'End [Market Spec Fetch] {len(self)} Stocks / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

    @classmethod
    def _format(cls, num) -> Union[int, float]:
        if not num:
            return nan
        try:
            return float(num) if "." in num else int(num)
        except ValueError:
            num = "".join([c for c in num if c.isdigit() or c in [".", "-"]])
            if not num or num == "." or num == "-":
                return nan
            return float(num) if "." in num else int(num)

    @classmethod
    def fetchMarketCap(cls, date: str, market:str='ALL') -> DataFrame:
        try:
            df = get_market_cap_by_ticker(date=date, market=market, alternative=True) \
                .rename(columns=CAP_LABEL)
            df.index.name = 'ticker'
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(CAP_LABEL.values()))

    @classmethod
    def fetchIpoList(cls) -> DataFrame:
        _url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download'
        try:
            resp = StringIO(get(_url).text)
            df = read_html(io=resp, encoding='euc-kr')[0][IPO_LABEL.keys()] \
                .rename(columns=IPO_LABEL) \
                .set_index(keys='ticker')
            df.index = df.index.astype(str).str.zfill(6)
            return df
        except (KeyError, RecursionError, JSONDecodeError, SSLError):
            return DataFrame(columns=list(IPO_LABEL.values()))

    @classmethod
    def fetchXml(cls, ticker: str, debug: bool = False) -> Union[str, Element]:
        resp = get(url=f"http://cdn.fnguide.com/SVO2/xml/Snapshot_all/{ticker}.xml")
        resp.encoding = 'euc-kr'
        text = resp.text.replace("<![CDATA[", "").replace("]]>", "")
        text = sub(r'<business_summary>.*?</business_summary>', '', text, flags=DOTALL)
        return text if debug else fromstring(text)

    @classmethod
    def fetchOverview(cls, xml: Element) -> Series:
        data = {}
        for key, tag in OVERVIEW_TAG.items():
            ftag = xml.find(tag)
            data[key] = None if ftag is None else ftag.text
        return Series(data=data).apply(cls._format)

    @classmethod
    def fetchStatement(cls, ticker_or_xml: Union[str, Element]) -> Tuple[DataFrame, DataFrame]:
        xml = cls.fetchXml(ticker_or_xml) if isinstance(ticker_or_xml, str) else ticker_or_xml

        def _statement(tag: str) -> DataFrame:
            obj = xml.find(tag)
            if obj is None:
                return DataFrame()
            columns = [val.text for val in obj.findall('field')]
            index, data = [], []
            for record in obj.findall('record'):
                index.append(record.find('date').text)
                data.append([val.text for val in record.findall('value')])
            df = DataFrame(index=index, columns=columns, data=data)
            return df.map(cls._format)

        B_A = _statement(STATEMENT_TAG['separateAnnual'])
        D_A = _statement(STATEMENT_TAG['consolidateAnnual'])
        if B_A.empty or D_A.empty:
            return DataFrame(), DataFrame()
        if B_A.count().sum() > D_A.count().sum():
            A = B_A
            Q = _statement(STATEMENT_TAG['separateQuarter'])
        else:
            A = D_A
            Q = _statement(STATEMENT_TAG['consolidateQuarter'])
        return A, Q

    @classmethod
    def customizeStatement(cls, statement: DataFrame, include_estimated: bool = False) -> DataFrame:
        st = statement.copy()
        if not include_estimated:
            st = st[~st.index.str.endswith('(E)')].copy()
        if st[st.index.str.endswith('(P)')].count().sum() == 0:
            st = st[~st.index.str.endswith('(P)')].copy()
        else:
            st.index = st.index.str.replace(r'\(P\)', '', regex=True)
        st['trailingRevenue'] = st[st.columns[0]].rolling(window=4, min_periods=1).sum()
        st['trailingProfit'] = st['영업이익(억원)'].rolling(window=4, min_periods=1).sum()
        st['trailingEps'] = st['EPS(원)'].rolling(window=4, min_periods=1).sum()
        st['trailingProfitRate'] = st['trailingProfit'] / st['trailingRevenue'] * 100
        st['revenueGrowth'] = 100 * (st[st.columns[0]].diff() / st[st.columns[0]].abs().shift(1))
        st['profitGrowth'] = 100 * (st['영업이익(억원)'].diff() / st['영업이익(억원)'].abs().shift(1))
        st['epsGrowth'] = 100 * (st['EPS(원)'].diff() / st['EPS(원)'].abs().shift(1))
        return st


if __name__ == "__main__":
    marketSpec = MarketSpec(True)
    # print(marketSpec)
    print(marketSpec.log)


Begin [Market Spec Fetch] @20250306
End [Market Spec Fetch] 1319 Stocks / Elapsed: 662.29s


In [ ]:
# 별도: 033100, 058470
# 연결: 005930, 000660, 005380, 316140
# Columns:
# ['매출액(억원)', '영업이익(억원)', '영업이익(발표기준)', '당기순이익(억원)', '  지배주주순이익(억원)',
#  '비지배주주순이익(억원)', '자산총계(억원)', '부채총계(억원)', '자본총계(억원)', '  지배주주지분(억원)',
#  '비지배주주지분(억원)', '자본금(억원)', '부채비율(%)', '유보율(%)', '영업이익률(%)',
#  '지배주주순이익률(%)', 'ROA(%)', 'ROE(%)', 'EPS(원)', 'BPS(원)', 'DPS(원)',
#  'PER(배)', 'PBR(배)', '발행주식수(천주)', '배당수익률(%)']
xml = MarketSpec.fetchXml('316140', debug=False)
# xml
ovv = MarketSpec.fetchOverview(xml)
a, q = MarketSpec.fetchStatement(xml)
Aa, Qq = MarketSpec.customizeStatement(a), MarketSpec.customizeStatement(q)

# ovv
# a
# Aa
# Qq

'<?xml version="1.0" encoding="euc-kr"?>\r\n<xml>\r\n<price>\r\n<date>2025/02/12</date>\r\n<close_val>16,850</close_val>\r\n<bef_close_val_calc>450</bef_close_val_calc>\r\n<high52week>17,200</high52week>\r\n<low52week>13,170</low52week>\r\n<change_1month>8.71</change_1month>\r\n<change_3month>4.59</change_3month>\r\n<change_6month>14.24</change_6month>\r\n<change_12month>15.33</change_12month>\r\n<change_36month>11.22</change_36month>\r\n<market_sum>125,127</market_sum>\r\n<listed_stock_1>742,591,501</listed_stock_1>\r\n<listed_stock_2>0</listed_stock_2>\r\n<deal_cnt>3,196,830</deal_cnt>\r\n<deal_amt>539</deal_amt>\r\n<frgn_rate>46.19</frgn_rate>\r\n<beta>0.74658</beta>\r\n<face_value>5,000</face_value>\r\n<ff_sher>654,893,082</ff_sher>\r\n<ff_sher_rt>88.19</ff_sher_rt>\r\n<mkt_cap_1>125,127</mkt_cap_1>\r\n<face_value_unit>원</face_value_unit>\r\n<face_value_chg_dt></face_value_chg_dt>\r\n</price>\r\n<credit_grade>\r\n<record>\r\n<year>2023</year>\r\n<grade>AAA</grade>\r\n</record>\r\n<

In [ ]:
# marketSpec
marketSpec.to_json(orient='index').replace('nan', '')

'{"005930":{"high52":87800.0,"low52":49900.0,"beta":1.2506,"floatShares":75.73,"estPrice":73520.0,"estEps":4328.0,"trailingRevenue":3008709.3399999999,"trailingEps":4950.0,"trailingProfitRate":10.88,"averageRevenueGrowth_A":6.16,"averageProfitGrowth_A":74.11,"averageEpsGrowth_A":33.99,"RevenueGrowth_A":16.2,"RevenueGrowth_Q":-4.19,"ProfitGrowth_A":398.34,"ProfitGrowth_Q":-29.3,"EpsGrowth_A":132.29,"EpsGrowth_Q":-22.57,"fiscalDividendYield":1.84,"fiscalDebtRatio":25.36},"000660":{"high52":241000.0,"low52":146800.0,"beta":1.7359,"floatShares":74.08,"estPrice":272880.0,"estEps":33900.0,"trailingRevenue":661929.6,"trailingEps":27183.0,"trailingProfitRate":35.45,"averageRevenueGrowth_A":26.44,"averageProfitGrowth_A":75.37,"averageEpsGrowth_A":-5.85,"RevenueGrowth_A":102.02,"RevenueGrowth_Q":12.48,"ProfitGrowth_A":403.58,"ProfitGrowth_Q":14.98,"EpsGrowth_A":317.16,"EpsGrowth_Q":39.17,"fiscalDividendYield":0.85,"fiscalDebtRatio":87.52},"207940":{"high52":1174000.0,"low52":727000.0,"beta":0.48

## build

### service

#### baseline.py

In [6]:
from datetime import datetime
from numpy import nan
from pandas import DataFrame
from time import time
from typing import Any, Dict, List
if "PATH" not in globals():
    try:
        from ...common.path import PATH
        from ...fetch.market.state import MarketState
        from ...fetch.market.group import MarketGroup
        from ...fetch.market.spec import MarketSpec
    except ImportError:
        from src.common.path import PATH
        from src.fetch.market.state import MarketState
        from src.fetch.market.group import MarketGroup
        from src.fetch.market.spec import MarketSpec


METADATA = {
    'close': {
        'label': '종가',
        'description': '종가',
        'unit': '원',
		'round': 0,
    },
    'marketCap': {
        'label': '시가총액',
        'description': '시가총액',
        'unit': '억원',
		'round': 0,
    },
    'volume': {
        'label': '거래량',
        'description': '거래량',
        'unit': '',
		'round': 0,
    },
    'amount': {
        'label': '거래대금',
        'description': '거래대금',
        'unit': '원',
        'round': 0
    },
    'PBR': {
        'label': 'PBR',
        'description': '주가 장부가(분기)',
        'unit': '',
        'round': 2
    },
    'dividendYield': {
        'label': '예상 배당수익률',
        'description': '직전 주당 배당금(DPS) / 현재가',
        'unit': '%',
		'round': 2,
    },
    'foreignRate': {
        'label': '외인보유율',
        'description': '외국인 보유 지분율',
        'unit': '%',
		'round': 2,
    },
    'D-1': {
        'label': '1일 수익률',
        'description': '-1D 수익률',
        'unit': '%',
		'round': 2,
    },
    'W-1': {
        'label': '1주 수익률',
        'description': '-1W 수익률',
        'unit': '%',
		'round': 2,
    },
    'M-1': {
        'label': '1개월 수익률',
        'description': '-1M 수익률',
        'unit': '%',
		'round': 2,
    },
    'M-3': {
        'label': '3개월 수익률',
        'description': '-3M 수익률',
        'unit': '%',
		'round': 2,
    },
    'M-6': {
        'label': '6개월 수익률',
        'description': '-6M 수익률',
        'unit': '%',
		'round': 2,
    },
    'Y-1': {
        'label': '1년 수익률',
        'description': '-1Y 수익률',
        'unit': '%',
		'round': 2,
    },
    'beta': {
        'label': '베타',
        'description': '주가 지수 대비 변동성 지표: = 1 (시장과 동일 비율 움직임), < 1 (시장보다 덜 움직임), > 1 (시장보다 큰 폭으로 움직임)',
        'unit': '',
		'round': 4,
    },
    'floatShares': {
        'label': '유동주식비율',
        'description': '전체 발행 주식 대비 시장 유동 주식 비율',
        'unit': '%',
		'round': 2,
    },
    'trailingRevenue': {
        'label': '매출액',
        'description': '연속 4분기에 대한 매출(영업수익) 합산',
        'unit': '억원',
		'round': 2,
    },
    'trailingEps': {
        'label': 'EPS',
        'description': '연속 4분기에 대한 EPS 합산',
        'unit': '원',
		'round': 2,
    },
    'trailingProfitRate': {
        'label': '영업이익률',
        'description': '연속 4분기 영업이익/매출에 대한 영업이익률',
        'unit': '%',
		'round': 2,
    },
    'averageRevenueGrowth_A': {
        'label': '연평균 매출성장율',
        'description': '연간 매출(영업수익) 성장율의 평균',
        'unit': '%',
		'round': 2,
    },
    'averageProfitGrowth_A': {
        'label': '연평균 영업이익성장율',
        'description': '연간 영업이익 성장율의 평균',
        'unit': '%',
		'round': 2,
    },
    'averageEpsGrowth_A': {
        'label': '연평균 EPS성장율',
        'description': '연간 EPS 성장율의 평균',
        'unit': '%',
		'round': 2,
    },
    'RevenueGrowth_A': {
        'label': '매출성장율(연간)',
        'description': '연간 매출(영업수익) 성장율',
        'unit': '%',
		'round': 2,
    },
    'RevenueGrowth_Q': {
        'label': '매출성장율(분기)',
        'description': '분기별 매출(영업수익) 성장율',
        'unit': '%',
		'round': 2,
    },
    'ProfitGrowth_A': {
        'label': '영업이익성장율(연간)',
        'description': '연간 영업이익 성장율',
        'unit': '%',
		'round': 2,
    },
    'ProfitGrowth_Q': {
        'label': '영업이익성장율(분기)',
        'description': '분기별 영업이익 성장율',
        'unit': '%',
		'round': 2,
    },
    'EpsGrowth_A': {
        'label': 'EPS성장율(연간)',
        'description': '연간 EPS 성장율',
        'unit': '%',
		'round': 2,
    },
    'EpsGrowth_Q': {
        'label': 'EPS성장율(분기)',
        'description': '분기별 EPS 성장율',
        'unit': '%',
		'round': 2,
    },
    'fiscalDividendYield': {
        'label': '배당수익률',
        'description': '직전 회계연도 기준 배당수익률',
        'unit': '%',
		'round': 2,
    },
    'fiscalDebtRatio': {
        'label': '부채율',
        'description': '직전 회계연도 기준 부채율',
        'unit': '%',
		'round': 2,
    },
    'pct52wHigh': {
        'label': '52주 최고가 대비',
        'description': '52주 최고가 대비 수익률',
        'unit': '%',
		'round': 2,
    },
    'pct52wLow': {
        'label': '52주 최저가 대비',
        'description': '52주 최저가 대비 수익률',
        'unit': '%',
		'round': 2,
    },
    'pctEstimated': {
        'label': '목표가 대비',
        'description': '목표가 대비 수익률(목표가 존재 시)',
        'unit': '%',
		'round': 2,
    },
    'estimatedPE': {
        'label': 'Forward PE',
        'description': '현재가 / 12개월 선행 EPS',
        'unit': '',
		'round': 2,
    },
    'trailingPS': {
        'label': 'PSR',
        'description': '주가 매출 비율(4분기 합산)',
        'unit': '',
		'round': 2,
    },
    'trailingPE': {
        'label': 'PER',
        'description': '주가 수익 비율(4분기 합산)',
        'unit': '',
		'round': 2,
    },
    'turnoverRatio': {
        'label': "거래회전율",
        'description': '시가총액 대비 거래대금 비율',
        'unit': '%',
        'round': 2,
    },
    'market': {
        'label': '시장구분',
        'description': '시장 구분',
        'unit': '',
        'round': -1
    },
    'name': {
        'label': '종목명',
        'description': '종목명',
        'unit': '',
		'round': -1,
    },
    'industryCode': {
        'label': '산업코드',
        'description': '산업코드',
        'unit': '',
		'round': -1,
    },
    'industryName': {
        'label': '산업명',
        'description': '산업 분류명',
        'unit': '',
		'round': -1,
    },
    'sectorCode': {
        'label': '섹터코드',
        'description': '섹터코드',
        'unit': '',
		'round': -1,
    },
    'sectorName': {
        'label': '섹터명',
        'description': '섹터 분류명',
        'unit': '',
		'round': -1,
    },
    'stockSize': {
        'label': '대형주여부',
        'description': '대형주 여부',
        'unit': '',
		'round': -1,
    },
    'date': {
        'label': '날짜',
        'description': '최근 거래일',
        'unit': '',
		'round': -1,
    },
}



class MarketBaseline(DataFrame):

    _log: List[str] = []
    meta: Dict[str, Dict[str, Any]] = METADATA.copy()
    def __init__(self, update:bool=True):
        stime = time()
        self.log = f'Begin [Build Market Baseline] @{datetime.today().strftime("%Y-%m-%d")[2:]}'

        spec = MarketSpec(update=False)
        group = MarketGroup(update=False)
        state = MarketState(update=update)
        merge = state.join(spec).join(group)
        if state.log:
            self.log = state.log

        try:
            merge['high52'] = merge[['close', 'high52']].max(axis=1)
            merge['low52'] = merge[['close', 'low52']].min(axis=1)
            merge['pct52wHigh'] = 100 * (merge['close'] / merge['high52'] - 1)
            merge['pct52wLow'] = 100 * (merge['close'] / merge['low52'] - 1)
            merge['pctEstimated'] = 100 * (merge['close'] / merge['estPrice'] - 1)
            merge['estEps'] = merge['estEps'].apply(lambda x: x if x > 0 else nan)
            merge['estimatedPE'] = merge['close'] / merge['estEps']
            merge.drop(columns=["high52", "low52", "estPrice", "estEps"], inplace=True)

            merge['trailingEps'] = merge['trailingEps'].apply(lambda x: x if x > 0 else nan)
            merge['trailingPS'] = (merge['marketCap'] / 1e+8) / merge['trailingRevenue']
            merge['trailingPE'] = merge['close'] / merge['trailingEps']
            merge['turnoverRatio'] = 100 * merge['amount'] / merge['marketCap']
            self.log = f'- No-Labeled Keys: {[key for key in merge if key not in METADATA]}'

            merge = merge[METADATA.keys()]
            for key, meta in METADATA.items():
                if not meta['round'] == -1:
                    merge[key] = round(merge[key], meta['round'])
        except Exception as report:
            self.log = f" * Error while customizing data: {report}"

        super().__init__(merge)
        self.log = f'End [Build Market Baseline] {len(self)} Stocks / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)


if __name__ == "__main__":
    from pandas import set_option

    set_option('display.expand_frame_repr', False)

    baseline = MarketBaseline(True)
    print(baseline)
    print(baseline.log)
    # print(baseline.columns)
    # print(baseline.loc[['005930', '005380']])


          close        marketCap   volume        amount   PBR  dividendYield  foreignRate   D-1    W-1    M-1  ...  trailingPE  turnoverRatio  market      name  industryCode  industryName  sectorCode  sectorName  stockSize        date
ticker                                                                                                         ...                                                                                                                        
005930    54100  320252411580200  7654455  415201706300  1.04           2.67        50.38  0.19  -3.91   2.66  ...       10.93           0.13   kospi      삼성전자         WI620           반도체         G45          IT      large  2025/03/06
000660   191600  139485253134000  1633825  314478602650  2.46           0.63        55.31  0.94   4.13  -0.10  ...        7.05           0.23   kospi    SK하이닉스         WI620           반도체         G45          IT      large  2025/03/06
207940  1128000   80284272000000    24065   27284746500  8.1

In [7]:
baseline.to_json(orient='index').replace('nan', '')

'{"005930":{"close":54100,"marketCap":320252411580200,"volume":7654455,"amount":415201706300,"PBR":1.04,"dividendYield":2.67,"foreignRate":50.38,"D-1":0.19,"W-1":-3.91,"M-1":2.66,"M-3":0.74,"M-6":-21.59,"Y-1":-25.79,"beta":1.2578,"floatShares":75.67,"trailingRevenue":3008709.0299999998,"trailingEps":4950.0,"trailingProfitRate":10.88,"averageRevenueGrowth_A":7.01,"averageProfitGrowth_A":85.24,"averageEpsGrowth_A":37.15,"RevenueGrowth_A":16.2,"RevenueGrowth_Q":-4.19,"ProfitGrowth_A":398.34,"ProfitGrowth_Q":-29.3,"EpsGrowth_A":132.29,"EpsGrowth_Q":-22.57,"fiscalDividendYield":2.72,"fiscalDebtRatio":27.93,"pct52wHigh":-38.38,"pct52wLow":8.42,"pctEstimated":-26.41,"estimatedPE":12.4,"trailingPS":1.06,"trailingPE":10.93,"turnoverRatio":0.13,"market":"kospi","name":"\\uc0bc\\uc131\\uc804\\uc790","industryCode":"WI620","industryName":"\\ubc18\\ub3c4\\uccb4","sectorCode":"G45","sectorName":"IT","stockSize":"large","date":"2025\\/03\\/06"},"000660":{"close":191600,"marketCap":139485253134000,"vo

#### marketmap.py

In [ ]:
class MarketMap(DataFrame):
    pass

#### bubble.py

In [ ]:
class MarketBubble(DataFrame):
    pass

### market.py

In [ ]:
pass

### cache.py

In [ ]:
pass

# Experiment

In [ ]:
# xml = MarketSpec.fetchXml('088980', True)
xml = MarketSpec.fetchXml('088980')
# MarketSpec.fetchOverview(xml)
a, q = MarketSpec.fetchStatement(xml, False)
# print(xml)
q

Empty DataFrame
Columns: []
Index: []


IndexError: single positional indexer is out-of-bounds